In [1]:
# 1
!uv --version

uv 0.9.5


In [2]:
# notes: the uv project already create and init via terminal
%cd uv-project

/workspaces/machine-learning-zoomcamp-homework/05-deployment/uv-project


In [3]:
!uv add scikit-learn==1.6.1

Using CPython 3.12.1 interpreter at: /home/codespace/.python/current/bin/python3.12
Creating virtual environment at: .venv
Resolved 6 packages in 26ms                                          
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 381ms                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.2
 + threadpoolctl==3.6.0


In [4]:
!grep -A 10 "scikit-learn" uv.lock

name = "scikit-learn"
version = "1.6.1"
source = { registry = "https://pypi.org/simple" }
dependencies = [
    { name = "joblib" },
    { name = "numpy" },
    { name = "scipy" },
    { name = "threadpoolctl" },
]
sdist = { url = "https://files.pythonhosted.org/packages/9e/a5/4ae3b3a0755f7b35a280ac90b28817d1f380318973cff14075ab41ef50d9/scikit_learn-1.6.1.tar.gz", hash = "sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e", size = 7068312, upload-time = "2025-01-10T08:07:55.348Z" }
wheels = [
--
    { name = "scikit-learn" },
]

[package.metadata]
requires-dist = [{ name = "scikit-learn", specifier = "==1.6.1" }]


In [15]:
# 3
import pickle
import pandas as pd
import json

def load_model(bin_file_path):
    try:
        with open(bin_file_path, 'rb') as file:
            model = pickle.load(file)
        print(f"✅ Model loaded successfully from {bin_file_path}")
        return model
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None

model = load_model('pipeline_v1.bin')  # or whatever your file is called
model

✅ Model loaded successfully from pipeline_v1.bin


,steps,"[('dictvectorizer', ...), ('logisticregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,True
,sort,True
,penalty,'l2'
,dual,False
,tol,0.0001


In [19]:
# Score the record
def score_record(model, record):
    try:
        # DictVectorizer expects a list of dictionaries
        # Even for a single record, it needs to be in a list
        input_data = [record]
        
        print("📊 Input data:")
        print(input_data)
        print("\n")
        
        # Make prediction
        prediction = model.predict(input_data)
        print(f"🎯 Prediction: {prediction[0]}")
        
        # Get probability scores
        probability = model.predict_proba(input_data)
        print(f"\n📈 Probability scores:")
        print(f"   - Class 0 (not convert): {probability[0][0]:.4f} ({probability[0][0]*100:.2f}%)")
        print(f"   - Class 1 (convert): {probability[0][1]:.4f} ({probability[0][1]*100:.2f}%)")
        
        return prediction[0], probability[0]
        
    except Exception as e:
        print(f"❌ Error scoring record: {type(e).__name__}: {e}")
        return None, None

# The record to score
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# Score it - pass the record as a list of dictionaries
prediction, probability = score_record(model, record)

📊 Input data:
[{'lead_source': 'paid_ads', 'number_of_courses_viewed': 2, 'annual_income': 79276.0}]


🎯 Prediction: 1

📈 Probability scores:
   - Class 0 (not convert): 0.4664 (46.64%)
   - Class 1 (convert): 0.5336 (53.36%)


In [4]:
# 4 (the api already set up at folder uv-project and already run the api service)
import requests

# The record to score
client = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# Make request to the API
url = "http://localhost:8000/predict"
response = requests.post(url, json=client)

# Print the result
print("Status Code:", response.status_code)
print("\nResponse:")
print(response.json())

Status Code: 200

Response:
{'prediction': 1, 'probability': {'not_convert': 0.46639272972019385, 'convert': 0.5336072702798061}}


In [8]:
# 6
import requests

url = "http://localhost:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
result = response.json()

print(f"Probability that lead will convert: {result['probability']['convert']:.2f}")

Probability that lead will convert: 0.99
